In [1]:
# Heatmaps and Grad-CAM
import os
import numpy as np
import tensorflow as tf
from IA.CNN.common.cam import get_img_array, make_gradcam_heatmap, display_heatmaps
from tensorflow.keras.models import load_model
import cv2
import matplotlib.pyplot as plt

In [2]:
from keras.applications.mobilenet_v2 import preprocess_input

subfolders = ['BLACK_ROT', 'ESCA', 'HEALTHY', 'LEAF_BLIGHT']

# Heatmap
images = [
    {
        "path": "../../../Datas/new/BLACK_ROT/0d907173-6ef8-4fbd-b682-f0a7a5263de0___FAM_B.Rot 3298.JPG"
    },
    {
        "path": "../../../Datas/new/ESCA/0db65255-6c76-4685-93fe-2967f9040e78___FAM_B.Msls 1176.JPG"
    },
    {
        "path": "../../../Datas/new/HEALTHY/1f9ead63-93e5-4379-885b-b2b57352202b___Mt.N.V_HL 8938.JPG"
    },
    {
        "path": "../../../Datas/new/LEAF_BLIGHT/0feed390-3065-47dc-881b-a9beb29ff312___FAM_L.Blight 1663.JPG"
    }
]

img_width, img_height = 160, 160

# Load the model
model = load_model('model.h5')

# Select last conv layer
last_conv_layer_name = "Conv_1"

# Remove last layer's softmax
model.layers[-1].activation = None

for image in images:
    heatmaps = []
    image_temp = cv2.imread(image["path"])
    image_temp = cv2.resize(image_temp, (img_width, img_height))

    plt.imshow(cv2.cvtColor(image_temp, cv2.COLOR_BGR2RGB))
    plt.show()

    image["array"] = get_img_array(image["path"], size=(img_width, img_height))
    
    # Préparez l'image pour le modèle
    image_preprocessed = preprocess_input(image["array"])

    # Faites une prédiction
    predictions = model.predict(image_preprocessed)

    # Obtenez l'indice de la classe prédite
    predicted_index = np.argmax(predictions[0])

    # Imprimez la classe prédite
    print(f"Predicted class: {subfolders[predicted_index]}")

    for layer in model.layers:
        if 'Conv' in layer.__class__.__name__:
            try: # Use try/except block to handle layers that can't be handled by Grad-CAM
                heatmap = make_gradcam_heatmap(image["array"], model, layer.name)
                if np.isnan(heatmap).any():
                    print(f'Heatmap for layer {layer.name} contains NaN values')
                else:
                    heatmaps.append({
                        "layer_name": layer.name,
                        "image": heatmap
                    })
            except Exception as e:
                print(f'Failed to generate Grad-CAM for layer {layer.name}. Reason: {str(e)}')

    # Display heatmaps from all layers until the last max pooling layer
    display_heatmaps(image["path"], heatmaps)

    # Display the chosen image superimposed with heatmaps of each layer until the last max pooling layer
    display_heatmaps(image["path"], heatmaps, True)



KeyError: 'array'